In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Modifying Design Variables, Constraints, and Objectives

OpenMDAO provides 3 methods to let the user modify options for design variables, constraints, and objectives after they were set in their respective methods `add_design_var`, `add_constraint`, and `add_objective`. The methods are:

```{eval-rst}
    .. automethod:: openmdao.core.system.System.set_design_var_options
        :noindex:
```

```{eval-rst}
    .. automethod:: openmdao.core.system.System.set_constraint_options
        :noindex:
```

```{eval-rst}
    .. automethod:: openmdao.core.system.System.set_objective_options
        :noindex:
```

## Example of Modifying Design Variable, Constraint, and Objective Options

Here is a simple examples of how these methods can be used for an optimizer example. Let's say a user runs an optimization, and then based on the result, they want to change the bounds of the constraint, and then re-optimize.

In [ ]:
# The initial optimization run

import numpy as np
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())
model = prob.model
model.nonlinear_solver = om.NonlinearBlockGS()

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
prob.driver.options['tol'] = 1e-9

model.add_design_var('z', lower=np.array([-10.0, 0.0]), upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

prob.setup()
prob.set_solver_print(level=0)
prob.run_driver()
print(f"con1 = {prob.get_val('con1')}")

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(prob.get_val('x'), 0.0, 1e-5)
assert_near_equal(prob.get_val('y1'), 3.160000, 1e-2)
assert_near_equal(prob.get_val('y2'), 3.755278, 1e-2)
assert_near_equal(prob.get_val('z'), [1.977639, 0.000000], 1e-2)
assert_near_equal(prob.get_val('obj'), 3.18339395, 1e-2);
assert_near_equal(prob.get_val('con1'), 0.0, 1e-2);

In [ ]:
# modify the constraint and run again

model.set_constraint_options(name='con1', upper=-1.0)
prob.setup()
prob.run_driver()
print(f"con1 = {prob.get_val('con1')}")

In [ ]:
assert_near_equal(prob.get_val('x'), 0.0, 1e-5)
assert_near_equal(prob.get_val('y1'), 4.16, 1e-2)
assert_near_equal(prob.get_val('y2'), 4.27921561, 1e-2)
assert_near_equal(prob.get_val('z'), [2.23960781e+00, 0.000000], 1e-2)
assert_near_equal(prob.get_val('obj'), 4.17385352, 1e-2);
assert_near_equal(prob.get_val('con1'), -1.0, 1e-2);